In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

num_epochs = 20
batch_size = 32
learning_rate = 0.001

In [2]:
def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)            # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [224, 224]) / 255.0
    return image_resized, label

data_text = pd.read_csv('train_data.csv')
# 构建训练数据集
train_filenames = tf.constant([filename for filename in list(data_text.path.values)])
train_labels = list(data_text.value.values)

train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
train_dataset = train_dataset.map(
    map_func=_decode_and_resize, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
# 取出前buffer_size个数据放入buffer，并从其中随机采样，采样后的数据用后续数据替换
train_dataset = train_dataset.shuffle(buffer_size=500)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [3]:
resnet50_fine_tune = tf.keras.models.Sequential()

resnet50_fine_tune.add(tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
resnet50_fine_tune.add(tf.keras.layers.Dense(2, activation = 'softmax'))
resnet50_fine_tune.layers[0].trainable = False
# resnet50_fine_tune.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])
resnet50_fine_tune.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
resnet50_fine_tune.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [4]:
resnet50_fine_tune.fit(train_dataset, epochs=num_epochs)

Epoch 1/20
154/154 [==============================] - 15s 96ms/step - loss: 0.7022 - sparse_categorical_accuracy: 0.5018
Epoch 2/20
154/154 [==============================] - 15s 95ms/step - loss: 0.6898 - sparse_categorical_accuracy: 0.5267
Epoch 3/20
154/154 [==============================] - 15s 95ms/step - loss: 0.6786 - sparse_categorical_accuracy: 0.5657
Epoch 4/20
154/154 [==============================] - 15s 94ms/step - loss: 0.6700 - sparse_categorical_accuracy: 0.5757
Epoch 5/20
154/154 [==============================] - 15s 95ms/step - loss: 0.6652 - sparse_categorical_accuracy: 0.6034
Epoch 6/20
154/154 [==============================] - 15s 95ms/step - loss: 0.6557 - sparse_categorical_accuracy: 0.6205
Epoch 7/20
154/154 [==============================] - 15s 95ms/step - loss: 0.6501 - sparse_categorical_accuracy: 0.6355
Epoch 8/20
154/154 [==============================] - 15s 95ms/step - loss: 0.6498 - sparse_categorical_accuracy: 0.6197
Epoch 9/20
154/154 [============